<a href="https://colab.research.google.com/github/Ilvecho/Project_7/blob/main/Project_7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns
import os
import pickle
import re
import random

import torch
from torch import nn
from torch import flatten
from torch.utils.data import TensorDataset, DataLoader
from torch.optim import Adam

from google.colab import files,drive
drive.mount('/content/gdrive')

# get the GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Mounted at /content/gdrive


In [100]:
data = pd.read_csv('/content/gdrive/MyDrive/P7_files/SalesCRM - CRM.csv')

In [101]:
for col in data.columns:
  print(col)

ID
Country
Education
First Contact
Last Contact
Status
Stage
First Call
Signed up for a demo
Filled in customer survey
Did sign up to the platform
Account Manager assigned
Subscribed


In [102]:
# Many of the cells have value NaN. Let's replace it with a different value so that we are able to count the number of NaN's as well
parsed_data = data.copy()
parsed_data['Country'] = parsed_data['Country'].fillna('Missing')
parsed_data['Education'] = parsed_data['Education'].fillna('Missing')
parsed_data['Status'] = parsed_data['Status'].fillna('Missing')
parsed_data['Stage'] = parsed_data['Stage'].fillna('Missing')


parsed_data.set_index('ID', inplace=True)

let's first analyze the **categorical** features

# Exploratory Data Analysis - Country

In [103]:
# Bar plot for the counties
fig = px.bar(
    parsed_data,
    x=parsed_data.groupby('Country').size().values,
    y=parsed_data.groupby('Country').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Country',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

I did not expect these many values for the country. Let's analyze this more in details:

In [104]:
parsed_data['Country'].value_counts()

USA                         6604
Missing                      891
Canada                       817
France                       336
UK                           329
                            ... 
uSA                            1
Hong Kong                      1
Singapore                      1
Cameroon                       1
Central African Republic       1
Name: Country, Length: 104, dtype: int64

There are **104** different Country values.
Actually there are some typos (e.g. uSA), so let's elaborate a bit the strings to get a more informative overview

In [105]:
countries = parsed_data['Country'].str.rstrip().str.lower()
countries.value_counts()

usa                         6641
missing                      891
canada                       818
france                       336
uk                           331
                            ... 
full time                      1
bulgaria                       1
senegal                        1
jordan                         1
central african republic       1
Name: Country, Length: 94, dtype: int64

The modification showed indeed some typos. \

We also added a **rstrip** function to remove trailing spaces.

The new count is **94**

In [106]:
countries.value_counts().tail(30)

bolivia                     2
kenya                       2
malaysia                    2
sri lanka                   2
philadelphia                1
congo                       1
california                  1
czechia                     1
russia&ukraine              1
nottingham                  1
korea                       1
bulgaria & uk               1
vietnamese                  1
greek                       1
romania                     1
turkey                      1
latvia                      1
hong kong                   1
seoul                       1
venezuela                   1
chuang                      1
benin                       1
guinea                      1
cameroon                    1
serbia                      1
full time                   1
bulgaria                    1
senegal                     1
jordan                      1
central african republic    1
Name: Country, dtype: int64

Looking at the tail, we still notice some duplicates (non compehensive list):
- Czechia is mentioned twice
- Bulgaria
- Nottingham is a city, not a country. Same is true for Seul, philadelphia and california
- Vietnamese is not a country
- Full time is not a country

Hence, let's create a lookup table to pre-process the country column

In [107]:
lookup_table = {
    "nottingham": "uk",
    "seoul": "south korea",
    "greek": "greece",
    "vietnamese": "vietnam",
    "bulgaria & uk": "bulgaria",
    "korea": "south korea",
    "russia&ukraine": "russia",
    "california": "usa",
    "philadelphia": "usa",
    "ca": "usa",
    'england': "uk",
    "dubai": "united arab emirates",
    "czechia (czech republic)": "czechia",
    "chuang": "china",
    "-": "missing"
}

In [108]:
countries = countries.apply(lambda x: lookup_table[x] if x in lookup_table.keys() else x)
countries.value_counts()

usa                         6645
missing                      901
canada                       818
france                       336
uk                           334
                            ... 
benin                          1
cameroon                       1
full time                      1
turkey                         1
central african republic       1
Name: Country, Length: 79, dtype: int64

Now we have **79** unique values

In [109]:
parsed_data['Country'] = countries.copy()

# Bar plot for the counties
fig = px.bar(
    parsed_data,
    x=countries.value_counts(),
    y=countries.value_counts().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Country',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

# Exploratory Data Analysis - Education

In [110]:
# Bar plot for the counties
fig = px.bar(
    parsed_data,
    x=parsed_data.groupby('Education').size().values,
    y=parsed_data.groupby('Education').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Education',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

the education category has a limited number of options - good.

Still important to notice that a lot of samples have value **missing**

# Exploratory Data Analysis - Status

The **Status** column, formerly "Message State", indicates how many times a potential customer was contacted. If the customer at some point is interested, they will proceed with the demo call

In [111]:
fig = px.bar(
    parsed_data,
    x=parsed_data.groupby('Status').size().values,
    y=parsed_data.groupby('Status').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Status',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

# Exploratory Data Analysis - Stage

The column stage reflects the potential customer reaction before and/or after the demo call

In [112]:
fig = px.bar(
    parsed_data,
    x=parsed_data.groupby('Stage').size().values,
    y=parsed_data.groupby('Stage').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Stage',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

# Date features

All the other features represent a date in time, so we need convert the columns to **datetime format**.

Let's define a parser function

In [113]:
# define the datetime parser function
def datetime_parser(column):

  # Iterate through all the rows, because different rows can have different datetime patterns
  for i in list(column.index):

    # Define the patterns observed in the data
    pattern_1 = re.compile(r'^[0-9]{4}\-[0-9]{2}\-[0-9]{4}$')
    pattern_2 = re.compile(r'^[0-9]{2}\.[0-9]{2}\.[0-9]{4}$')
    pattern_3 = re.compile(r'^[0-9]{2}\-[0-9]{2}\-[0-9]{4}$')

    # If the row is already in Datetime format, pass
    if isinstance(column.loc[i], pd.Timestamp):
      pass

    # If the value is missing, assign Not a Time
    elif pd.isna(column.loc[i]):
      column.loc[i] = pd.NaT

    # If the string matches the first patter, convert accordingly
    elif pattern_1.match(column.loc[i]):
      column.loc[i] = pd.to_datetime(column.loc[i], format='%Y-%m-%d')

    # If the string matches the second patter, convert accordingly
    elif pattern_2.match(column.loc[i]):
      column.loc[i] = pd.to_datetime(column.loc[i], format='%d.%m.%Y')

    # If the string matches the third patter, we are still not sure about the format
    elif pattern_3.match(column.loc[i]):
      # So, try one format and catch exceptions and use the other possible format
      try:
        column.loc[i] = pd.to_datetime(column.loc[i], format='%d-%m-%Y')
      except:
        column.loc[i] = pd.to_datetime(column.loc[i], format='%m-%d-%Y')

  return pd.to_datetime(column)

Manually fix an exception

In [114]:
# There is one row that has format Y-d-m rather then Y-m-d
parsed_data['First Contact'].iloc[9737] = "2021-12-13"
parsed_data['Last Contact'].iloc[9737] = "2021-12-13"

Parse the **First Contact** column

In [115]:
parsed_data['First Contact'] = datetime_parser(parsed_data['First Contact'])

Parse the **Last Contact** column

In [ ]:
parsed_data['Last Contact'] = datetime_parser(parsed_data['Last Contact'])

Parse the **First Call** column

In [ ]:
parsed_data['First Call'] = datetime_parser(parsed_data['First Call'])

Parse the **Signed up for a demo** column

In [118]:
parsed_data['Signed up for a demo'] = datetime_parser(parsed_data['Signed up for a demo'])

Parse the **Filled in customer survey** column

In [ ]:
parsed_data['Filled in customer survey'] = datetime_parser(parsed_data['Filled in customer survey'])

Parse the **Did sign up to the platform** column.

Actually we notice that some cells in this column do not contain a datetime value, but rather the string "No"

In [120]:
np.where(parsed_data['Did sign up to the platform'] == 'No')

(array([ 301,  382,  445,  446,  473,  610, 1028, 1052, 1087, 1232, 1334,
        1376, 1427, 1472, 1571, 1623, 1660, 1712, 1747, 1854, 2010, 2012,
        2021, 2023, 2261, 2286, 2439, 2541, 2643, 2650, 2902, 3010, 3081,
        3225, 3253, 3254, 3266, 3470, 3497, 3528, 3538, 3613, 3638, 3709,
        3765, 3871, 3951, 3984, 4012, 4042, 4046, 4061, 4177, 4178, 4222,
        4234, 4241, 4273, 4279, 4288, 4294, 4411, 4657, 4669, 4724, 4755,
        4758, 4759, 4805, 4860, 4891, 4916, 5002, 5169, 5177, 5185, 5249,
        5251, 5258, 5274, 5275, 5278, 5299, 5300, 5329, 5397, 5497, 5499,
        5648, 5688, 5691, 5694, 5787, 5942, 5943, 5951, 5966, 6004, 6116,
        6245, 6287, 6289, 6373, 6375, 6377, 6527, 6529, 6565, 6572, 6575,
        6642, 6646, 6728, 6759, 6903, 6905, 6913, 6915, 6950, 6984, 7006,
        7052, 7101, 7102, 8085, 8352, 8354]),)

We expect that customers that did not signed up to the platform are **not interested** in the product.

Hence, we need to make sure that the "Stage" column reflects this

In [121]:
did_not_signed = parsed_data.iloc[[ 301,  382,  445,  446,  473,  610, 1028, 1052, 1087, 1232, 1334,
        1376, 1427, 1472, 1571, 1623, 1660, 1712, 1747, 1854, 2010, 2012,
        2021, 2023, 2261, 2286, 2439, 2541, 2643, 2650, 2902, 3010, 3081,
        3225, 3253, 3254, 3266, 3470, 3497, 3528, 3538, 3613, 3638, 3709,
        3765, 3871, 3951, 3984, 4012, 4042, 4046, 4061, 4177, 4178, 4222,
        4234, 4241, 4273, 4279, 4288, 4294, 4411, 4657, 4669, 4724, 4755,
        4758, 4759, 4805, 4860, 4891, 4916, 5002, 5169, 5177, 5185, 5249,
        5251, 5258, 5274, 5275, 5278, 5299, 5300, 5329, 5397, 5497, 5499,
        5648, 5688, 5691, 5694, 5787, 5942, 5943, 5951, 5966, 6004, 6116,
        6245, 6287, 6289, 6373, 6375, 6377, 6527, 6529, 6565, 6572, 6575,
        6642, 6646, 6728, 6759, 6903, 6905, 6913, 6915, 6950, 6984, 7006,
        7052, 7101, 7102, 8085, 8352, 8354]]

fig = px.bar(
    did_not_signed,
    x=did_not_signed.groupby('Stage').size().values,
    y=did_not_signed.groupby('Stage').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Stage',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

The majority of the rows already have the column "Stage" indicating the lack of interest.

Let's manually fix the few rows in which this is not true

In [ ]:
for i in list(did_not_signed.index):
  if parsed_data['Stage'].loc[i] == 'interested' or parsed_data['Stage'].loc[i] == 'Missing':
    parsed_data['Stage'].loc[i] = 'not interested'

In [123]:
did_not_signed = parsed_data.iloc[[ 301,  382,  445,  446,  473,  610, 1028, 1052, 1087, 1232, 1334,
        1376, 1427, 1472, 1571, 1623, 1660, 1712, 1747, 1854, 2010, 2012,
        2021, 2023, 2261, 2286, 2439, 2541, 2643, 2650, 2902, 3010, 3081,
        3225, 3253, 3254, 3266, 3470, 3497, 3528, 3538, 3613, 3638, 3709,
        3765, 3871, 3951, 3984, 4012, 4042, 4046, 4061, 4177, 4178, 4222,
        4234, 4241, 4273, 4279, 4288, 4294, 4411, 4657, 4669, 4724, 4755,
        4758, 4759, 4805, 4860, 4891, 4916, 5002, 5169, 5177, 5185, 5249,
        5251, 5258, 5274, 5275, 5278, 5299, 5300, 5329, 5397, 5497, 5499,
        5648, 5688, 5691, 5694, 5787, 5942, 5943, 5951, 5966, 6004, 6116,
        6245, 6287, 6289, 6373, 6375, 6377, 6527, 6529, 6565, 6572, 6575,
        6642, 6646, 6728, 6759, 6903, 6905, 6913, 6915, 6950, 6984, 7006,
        7052, 7101, 7102, 8085, 8352, 8354]]

fig = px.bar(
    did_not_signed,
    x=did_not_signed.groupby('Stage').size().values,
    y=did_not_signed.groupby('Stage').size().index
)

fig.update_layout(
    xaxis_title='Count',
    yaxis_title='Stage',
    xaxis=dict(showline=False, showgrid=False),
    yaxis=dict(showline=False, showgrid=False)
)

fig.show()

Now we just need to remove the "No" strings from the column and parse it as datetime

In [129]:
parsed_data['Did sign up to the platform'] = parsed_data['Did sign up to the platform'].replace('No', pd.NaT)

# When running the parser we notice that one cell has a typo in the year: 20221
# let's find the row and fix the issue
np.where(parsed_data['Did sign up to the platform'] == '20221-08-24')

array([353])

In [ ]:
parsed_data['Did sign up to the platform'].iloc[353] = '2021-08-24'

In [ ]:
# Finally parse the column
parsed_data['Did sign up to the platform'] = datetime_parser(parsed_data['Did sign up to the platform'])

In [ ]:
parsed_data['Account Manager assigned'] = datetime_parser(parsed_data['Account Manager assigned'])

In [134]:
# Also column "subscribed" has an unexpected value - we need to manually modify it
np.where(parsed_data['Subscribed'] == '0000-00-00')

(array([7176]),)

In [ ]:
parsed_data['Subscribed'].iloc[7176] = pd.NaT

In [ ]:
parsed_data['Subscribed'] = datetime_parser(parsed_data['Subscribed'])

In [137]:
parsed_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11032 entries, 1 to 11793
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   Country                      11032 non-null  object        
 1   Education                    11032 non-null  object        
 2   First Contact                10317 non-null  datetime64[ns]
 3   Last Contact                 10722 non-null  datetime64[ns]
 4   Status                       11032 non-null  object        
 5   Stage                        11032 non-null  object        
 6   First Call                   441 non-null    datetime64[ns]
 7   Signed up for a demo         283 non-null    datetime64[ns]
 8   Filled in customer survey    184 non-null    datetime64[ns]
 9   Did sign up to the platform  296 non-null    datetime64[ns]
 10  Account Manager assigned     71 non-null     datetime64[ns]
 11  Subscribed                   47 non-null 